# imports

In [ ]:
import pandas as pd
import os
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt

In [ ]:
cwd

# setup

In [ ]:
cwd = os.getcwd()
print(cwd)

try:
    traffic = pd.read_parquet(os.path.join('..', 'data', 'heaton_traffic.parquet'))
    sensors = pd.read_parquet(os.path.join('..', 'data', 'heaton_sensors.parquet'))
    # add geometry to sensors
    sensors = gpd.GeoDataFrame(sensors, geometry=gpd.points_from_xy(sensors.lon, sensors.lat),crs="EPSG:4236")  # British National Grid
    print("----- Traffic Data -----")
    print(traffic.head(3))
    print("")
    print("----- Sensor Data -----")
    print(sensors.head(3))
    print("")


except FileNotFoundError:
    print("Error: Data files not found.")


try:
    heaton_polygon = gpd.read_file(os.path.join("..", "data", "heaton_ltn_polygon.gpkg"))
    print("----- Heaton LTN Polygon -----")
    print(heaton_polygon)

except FileNotFoundError:
    print("Error: Heaton LTN polygon file not found.")


In [ ]:
sensors.geometry.explore(marker_kwds={'radius': 10, 'color': 'red', 'fill': True, 'fillColor': 'red', 'fillOpacity': 0.6})

In [ ]:
# get street network 
G = ox.graph_from_bbox(
    north=54.994756,
    south=54.970025,
    west=-1.611626,
    east=-1.565620,
    network_type="all")



# add in/out of ltn tags to sensors

In [ ]:
# add default in_study_ltn column to sensors
sensors['in_study_ltn'] = False
mask = sensors.geometry.within(heaton_polygon.unary_union)
sensors.loc[mask, 'in_study_ltn'] = True
print(f"Marked {int(mask.sum())} sensors as in_study_ltn (total sensors: {len(sensors)})")

In [ ]:
# project
sensors_proj = sensors.to_crs("EPSG:27700")
heaton_poly_proj = heaton_polygon.to_crs("EPSG:27700").unary_union

buffers = {
    'within_50m': 50,
    'within_100m': 100,
    'within_500m': 500,
    'within_1km': 1000
}

# create columns 
for col in buffers:
    sensors_proj[col] = False

# assign True if the sensor point is within the buffer
for col, dist in buffers.items():
    buf = heaton_poly_proj.buffer(dist)
    sensors_proj[col] = sensors_proj.geometry.within(buf)


sensors[['within_50m','within_100m','within_500m','within_1km']] = sensors_proj[['within_50m','within_100m','within_500m','within_1km']]

print("-----Sensors-----")
print(sensors.head(3))


# get data within pre-during-post

Filters went in around October 2022. They came out around April 22 2024. See https://www.spaceforheaton.com/category/ltn/

In [ ]:
traffic['dt'] = pd.to_datetime(traffic['dt'])
cutoff = pd.Timestamp('2024-04-22')
traffic['period'] = traffic['dt'].apply(lambda x: 'during' if x < cutoff else 'post')

In [ ]:
traffic = traffic.merge(
    sensors[['location','within_50m']],
    on='location',
    how='left'
)

# ensure dt is datetime and date column exists
traffic['dt'] = pd.to_datetime(traffic['dt'])
traffic['date'] = traffic['dt'].dt.date

# group by period, date, veh_class, and within_50m and compute daily sum
daily = (
    traffic.groupby(['period', 'within_50m', 'date', 'veh_class'])['value']
    .sum()
    .reset_index()
)

# compute average daily value per veh_class, period, and within_50m
avg_daily = (
    daily.groupby(['period','within_50m','veh_class'])['value']
    .mean()
    .reset_index()
)

# pivot for plotting: veh_class as index, multi-columns (period + within_50m)
avg_pivot = avg_daily.pivot_table(
    index='veh_class', 
    columns=['period','within_50m'], 
    values='value'
)

# plot
avg_pivot.plot(kind='bar', figsize=(12,6))
plt.ylabel('Average Daily Value')
plt.title('Average Daily Traffic per Vehicle Class\nDuring vs Post, Within 50m vs Elsewhere')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
